## Analyse the stretch trajactory

#### PDKCP, stretch 150%

In [ ]:
import numpy as np
import copy
import matplotlib.pyplot as plt
from scipy import optimize

import psutil
import os
import gc

In [ ]:
def read_files(route,splited_trj,start_point):
    file = open(f"{route}","r")

    info_r1 = file.readlines()
    file.close()

    info_s1 = []
    for i in range (len(info_r1)):
        info_s1.append(info_r1[i].split())
    info_r1 = []
        

    info = psutil.virtual_memory()
    print(info.percent)
    
    ##   0% --- to --- 50%
    Natom_s1 = int(info_s1[3][0])
    index_s1 = []
    for i in range (len(info_s1)):
        if info_s1[i] == ['ITEM:', 'TIMESTEP']:
            index_s1.append(i)

    for j in range (int(start_point),len(index_s1)):
        frame = np.zeros((Natom_s1,5))
        for i in range (Natom_s1):
            frame[i,0] = int(info_s1[index_s1[j]+i+9][0])
            frame[i,1] = float(info_s1[index_s1[j]+i+9][1])
            frame[i,2] = float(info_s1[index_s1[j]+i+9][2])
            frame[i,3] = float(info_s1[index_s1[j]+i+9][3])
            frame[i,4] = float(info_s1[index_s1[j]+i+9][4])
        splited_trj.append([frame,info_s1[index_s1[j]+5:index_s1[j]+8]])
    
    return splited_trj

In [ ]:
def AD_dihedral(pos1,pos2,pos3,pos4):
    
    vec1 = np.zeros((1,3))
    vec2 = np.zeros((1,3))
    vec3 = np.zeros((1,3))
    
    # normal vector
    n1 = np.zeros((1,3))
    n2 = np.zeros((1,3))
    
    vec1[0,0] = pos1[0] - pos2[0]
    vec1[0,1] = pos1[1] - pos2[1]
    vec1[0,2] = pos1[2] - pos2[2]
    
    vec2[0,0] = pos3[0] - pos2[0]
    vec2[0,1] = pos3[1] - pos2[1]
    vec2[0,2] = pos3[2] - pos2[2]
    
    vec3[0,0] = pos4[0] - pos3[0]
    vec3[0,1] = pos4[1] - pos3[1]
    vec3[0,2] = pos4[2] - pos3[2]
    
    n1 = np.cross(vec1,vec2)
    n2 = np.cross(vec2,vec3)
    
    n1 = n1/np.sqrt(n1[0,0]**2+n1[0,1]**2+n1[0,2]**2)
    n2 = n2/np.sqrt(n2[0,0]**2+n2[0,1]**2+n2[0,2]**2)
    
    dihedral_angle = 180-np.arccos(np.dot(n1,np.transpose(n2)))/np.pi*180
    
    return dihedral_angle

In [ ]:
def get_plots(traj,strain):
    ## distribution
    N_index = []
    for i in range (len(traj[0][0])):
        if traj[0][0][i][1] == 14.007:
            N_index.append(i)

    angle_distribution = []
    for t in range (len(traj)):
        for i in range (int(len(N_index)/2)):
            angles = AD_dihedral(traj[t][0][N_index[2*i+1]-2][2:],traj[t][0][N_index[2*i+1]-1][2:],
                                 traj[t][0][N_index[2*i+1]][2:],traj[t][0][N_index[2*i+1]+1][2:])
            angle_distribution.append(angles[0][0])
    
    bins = 120
    interval = 1.5
    min_ang = 0
    max_ang = 180
    
    x_axis = np.linspace(min_ang,max_ang,bins+1)
    plot_ang = np.zeros((bins,2))
    
    for i in range (len(angle_distribution)):
        for j in range (bins):
            if angle_distribution[i] >= x_axis[j]-interval/2 and angle_distribution[i] < x_axis[j]+interval/2:
                plot_ang[j,1] += 1

    for i in range (bins):
        plot_ang[i,0] = x_axis[i]
        plot_ang[i,1] /= len(angle_distribution)    


    ## Fitting the curve
    mean = sum(plot_ang[:,0]*plot_ang[:,1])/bins
    sigma = sum(plot_ang[:,1]*(plot_ang[:,0]-mean)**2)/bins

    def gaus(x,a,x0,sigma,c):
        return a*np.exp(-(x-x0)**2/(2*sigma**2))+c

    popt,pcov = optimize.curve_fit(gaus,plot_ang[:,0],plot_ang[:,1],p0 = [1, 90, 1,0])

    plot_0 = copy.deepcopy(plot_ang)
    popt_0 = copy.deepcopy(popt)

    fig = plt.figure(figsize=(8,6))
    ax = fig.add_subplot(111)

    ax.scatter(plot_ang[:,0],plot_ang[:,1]*100,label='Data',linewidths=5)
    ax.plot(plot_ang[:,0],gaus(plot_ang[:,0],*popt)*100,color="red",label='fit',linewidth=6.0)
    ax.set_xlabel("Dihedral angle ($^\circ$)",size=20)
    ax.set_ylabel("Probability (%)",size=20)

    ax.set_title(f"strain = {strain}%",size=20)
    ax.set_xticks([0,30,60,90,120,150,180])
    ax.set_xlim(-4,184)
    
    
    ax.legend()

#     plt.savefig(f"{name}_{temperature}_{strain}%.pdf")
    plt.show()
    
    return plot_0,popt_0

In [ ]:
def comparsion(plot_0,plot_25,plot_50,plot_75,plot_100,popt_0,popt_25,popt_50,popt_75,popt_100):
    fig = plt.figure(figsize=(8,6))
    ax = fig.add_subplot(111)
#     ins = ax.inset_axes([105.4,25,22,22],transform=ax.transData)
    ins = ax.inset_axes([135.4,1.8,42,2.2],transform=ax.transData)

    ax.scatter(plot_0[:,0],plot_0[:,1]*100,label="0% strain",linewidths=1)
    ax.scatter(plot_25[:,0],plot_25[:,1]*100,label='25% strain',linewidths=1)
    ax.scatter(plot_50[:,0],plot_50[:,1]*100,label='50% strain',linewidths=1)
    ax.scatter(plot_75[:,0],plot_75[:,1]*100,label='75% strain',linewidths=1)
    ax.scatter(plot_100[:,0],plot_100[:,1]*100,label='100% strain',linewidths=1)

    ax.set_xlabel("Dihedral angle ($^\circ$)",size=15)
#     ax.set_ylabel("Counts ",size=15)
    ax.set_ylabel("Probability (%)",size=15)
    ax.set_title(f"{bname}",size=20)
    ax.set_xticks(np.arange(60,121,step=10))
    ax.set_xlim(-4,184)
    ax.set_ylim(-0.2,4.2)
#     ax.set_yticks(np.arange(0,4.1,step=1))
    ax.legend(loc="upper left",fontsize="x-large")
    
    def gaus(x,a,x0,sigma,c):
        return a*np.exp(-(x-x0)**2/(2*sigma**2))+c

    ins.plot(plot_0[:,0],gaus(plot_0[:,0],*popt_0)*100,label='fit_0%',linewidth=2)
    ins.plot(plot_25[:,0],gaus(plot_25[:,0],*popt_25)*100,label='fit_25%',linewidth=2)
    ins.plot(plot_50[:,0],gaus(plot_50[:,0],*popt_50)*100,label='fit_50%',linewidth=2)
    ins.plot(plot_75[:,0],gaus(plot_75[:,0],*popt_75)*100,label='fit_75%',linewidth=2)
    ins.plot(plot_100[:,0],gaus(plot_100[:,0],*popt_100)*100,label='fit_100%',linewidth=2)


    ins.set_xlabel("Dihedral angle ($^\circ$)",size=10)
    ins.set_xticks(np.arange(0,181,step=90))
    ins.set_ylim(-0.05,3.05)
    #ins.legend()

#     plt.savefig(f"{name}_{temperature}_all.pdf")
    plt.show()

In [ ]:
# def comparsion3(plot_0,plot_25,plot_50,popt_0,popt_25,popt_50):
#     fig = plt.figure(figsize=(8,6))
#     ax = fig.add_subplot(111)
# #     ins = ax.inset_axes([105.4,25,22,22],transform=ax.transData)
#     ins = ax.inset_axes([135.4,1.8,42,2.2],transform=ax.transData)

#     ax.scatter(plot_0[:,0],plot_0[:,1]*100,label="0% strain",linewidths=1)
#     ax.scatter(plot_25[:,0],plot_25[:,1]*100,label='25% strain',linewidths=1)
#     ax.scatter(plot_50[:,0],plot_50[:,1]*100,label='50% strain',linewidths=1)
# #     ax.scatter(plot_75[:,0],plot_75[:,1]*100,label='75% strain',linewidths=1)
# #     ax.scatter(plot_100[:,0],plot_100[:,1]*100,label='100% strain',linewidths=1)

#     ax.set_xlabel("Dihedral angle ($^\circ$)",size=15)
# #     ax.set_ylabel("Counts ",size=15)
#     ax.set_ylabel("Probability (%)",size=15)
#     ax.set_title(f"{bname}",size=20)
# #     ax.set_xticks(np.arange(60,121,step=10))
#     ax.set_xticks(np.arange(0,181,step=30))
#     ax.set_xlim(-4,184)
#     ax.set_ylim(-0.2,4.2)
# #     ax.set_yticks(np.arange(0,4.1,step=1))
#     ax.legend(loc="upper left",fontsize="x-large")
    
#     def gaus(x,a,x0,sigma,c):
#         return a*np.exp(-(x-x0)**2/(2*sigma**2))+c

#     ins.plot(plot_0[:,0],gaus(plot_0[:,0],*popt_0)*100,label='fit 0%',linewidth=2)
#     ins.plot(plot_25[:,0],gaus(plot_25[:,0],*popt_25)*100,label='fit 25%',linewidth=2)
#     ins.plot(plot_50[:,0],gaus(plot_50[:,0],*popt_50)*100,label='fit 50%',linewidth=2)
# #     ins.plot(plot_75[:,0],gaus(plot_75[:,0],*popt_75)*100,label='fit_75%',linewidth=2)
# #     ins.plot(plot_100[:,0],gaus(plot_100[:,0],*popt_100)*100,label='fit_100%',linewidth=2)


#     ins.set_xlabel("Dihedral angle ($^\circ$)",size=10)
#     ins.set_xticks(np.arange(0,181,step=90))
#     ins.set_ylim(-0.05,3.05)
#     #ins.legend()

#     plt.savefig(f"all.pdf")
#     plt.show()

# M

In [ ]:
name="pdkcm"
temperature = "1.2"
sname = "m"
bname = "PDKCM"

chain_numbers =44
d_of_p = 43 # degree of polymerization

# splited_trj1 = []
# splited_trj1 = read_files(f"s0-ta.trj",splited_trj1,0)
# plot_m_0,popt_m_0 = get_plots(splited_trj1,0)
# splited_trj1 = []

splited_trj2 = []
splited_trj2 = read_files(f"s7-ta.trj",splited_trj2,0)
plot_m_25,popt_m_25 = get_plots(splited_trj2,25)
splited_trj2 = []

splited_trj3 = []
splited_trj3 = read_files(f"s13_ta.trj",splited_trj3,0)
plot_m_50,popt_m_50 = get_plots(splited_trj3,50)
splited_trj3 = []

splited_trj4 = []
splited_trj4 = read_files(f"s18_ta.trj",splited_trj4,0)
plot_m_75,popt_m_75 = get_plots(splited_trj4,75)
splited_trj4 = []


splited_trj5 = []
splited_trj5 = read_files(f"s27_ta.trj",splited_trj5,0)
plot_m_100,popt_m_100 = get_plots(splited_trj5,100)
splited_trj5 = []

In [ ]:
comparsion(plot_m_0,plot_m_25,plot_m_50,plot_m_75,plot_m_100,popt_m_0,popt_m_25,popt_m_50,popt_m_75,popt_m_100)

In [ ]:
file = open(f"{name}_{temperature}_data.txt","w")

file.write("Angle"+"    ")
file.write("0%"+"    ")
file.write("25%"+"    ")
file.write("50%"+"    ")
file.write("75%"+"    ")
file.write("100%")
file.write("\n")

for i in range (len(plot_m_0)):
    file.write(f"{plot_m_0[i,0]}"+"    ")
    file.write("%.5f"%(plot_m_0[i,1])+" ")
    file.write("%.5f"%(plot_m_25[i,1])+" ")
    file.write("%.5f"%(plot_m_50[i,1])+" ")
    file.write("%.5f"%(plot_m_75[i,1])+" ")
    file.write("%.5f"%(plot_m_100[i,1]))
    file.write("\n")
    
file.close()

In [ ]:
file = open(f"{name}_{temperature}_fitting.txt","w")

file.write("Fitting format: a*exp(-(x-x0)^2/(2*sigma^2))+c")
file.write("\n")

file.write("Strain"+"    ")
file.write("a"+"    ")
file.write("x0"+"    ")
file.write("sigma"+"    ")
file.write("c"+"    ")
file.write("\n")

id_ = 0
for strain in [popt_m_0,popt_m_25,popt_m_50,popt_m_75,popt_m_100]:
    file.write(f"{id_*25}%"+"    ")
    id_ +=1
    for i in range (4):
        file.write("%.8f"%(strain[i])+"    ")
    file.write("\n")
    
file.close()

# P

In [ ]:
name="pdkcp"
temperature = "1.2"
sname = "p"
bname = "PDKCP"

chain_numbers = 38
d_of_p = 48 # degree of polymerization

splited_trj1 = []
splited_trj1 = read_files(f"s0-ta.trj",splited_trj1,0)
plot_p_0,popt_p_0 = get_plots(splited_trj1,0)
splited_trj1 = []

splited_trj2 = []
splited_trj2 = read_files(f"s7-ta.trj",splited_trj2,0)
plot_p_25,popt_p_25 = get_plots(splited_trj2,25)
splited_trj2 = []

splited_trj3 = []
splited_trj3 = read_files(f"s13_ta.trj",splited_trj3,0)
plot_p_50,popt_p_50 = get_plots(splited_trj3,50)
splited_trj3 = []

splited_trj4 = []
splited_trj4 = read_files(f"s18_ta.trj",splited_trj4,0)
plot_p_75,popt_p_75 = get_plots(splited_trj4,75)
splited_trj4 = []


splited_trj5 = []
splited_trj5 = read_files(f"s27_ta.trj",splited_trj5,0)
plot_p_100,popt_p_100 = get_plots(splited_trj5,100)
splited_trj5 = []

In [ ]:
comparsion(plot_p_0,plot_p_25,plot_p_50,plot_p_75,plot_p_100,popt_p_0,popt_p_25,popt_p_50,popt_p_75,popt_p_100)

In [ ]:
file = open(f"{name}_{temperature}_data.txt","w")

file.write("Angle"+"    ")
file.write("0%"+"    ")
file.write("25%"+"    ")
file.write("50%"+"    ")
file.write("75%"+"    ")
file.write("100%")
file.write("\n")

for i in range (len(plot_p_0)):
    file.write(f"{plot_p_0[i,0]}"+"    ")
    file.write("%.5f"%(plot_p_0[i,1])+" ")
    file.write("%.5f"%(plot_p_25[i,1])+" ")
    file.write("%.5f"%(plot_p_50[i,1])+" ")
    file.write("%.5f"%(plot_p_75[i,1])+" ")
    file.write("%.5f"%(plot_p_100[i,1]))
    file.write("\n")
    
file.close()

In [ ]:
file = open(f"{name}_{temperature}_fitting.txt","w")

file.write("Fitting format: a*exp(-(x-x0)^2/(2*sigma^2))+c")
file.write("\n")

file.write("Strain"+"    ")
file.write("a"+"    ")
file.write("x0"+"    ")
file.write("sigma"+"    ")
file.write("c"+"    ")
file.write("\n")

id_ = 0
for strain in [popt_p_0,popt_p_25,popt_p_50,popt_p_75,popt_p_100]:
    file.write(f"{id_*25}%"+"    ")
    id_ +=1
    for i in range (4):
        file.write("%.8f"%(strain[i])+"    ")
    file.write("\n")
    
file.close()

# H

In [ ]:
name="pdkch"
temperature = "1.2"
sname = "h"
bname="PDKCH"

chain_numbers = 46
d_of_p =  36 # degree of polymerization

splited_trj1 = read_files(f"stretch{temperature}/rerun2/{sname}/eq2/c_npt2.trj")

In [ ]:
plot_h_0,popt_h_0 = get_plots(splited_trj1,0,0)

In [ ]:
splited_trj1 = read_files(f"s1.trj")
# splited_trj2 = read_files(f"s9.trj")
# splited_trj3 = read_files(f"s16.trj")
# splited_trj3 = read_files(f"s21.trj")
# splited_trj3 = read_files(f"s26.trj")

In [ ]:
plot_h_0,popt_h_0 = get_plots(splited_trj1,0,0)
# plot_h_25,popt_h_25 = get_plots(splited_trj2,21,25)
# plot_h_50,popt_h_50 = get_plots(splited_trj3,-1,50)
# plot_h_75,popt_h_75 = get_plots(splited_trj4,-1,75)
# plot_h_100,popt_h_100 = get_plots(splited_trj5,-1,100)

In [ ]:
name="pdkch"
temperature = "1.2"
sname = "h"
bname="PDKCH"

comparsion(plot_h_0,plot_h_25,plot_h_50,plot_h_75,plot_h_100,popt_h_0,popt_h_25,popt_h_50,popt_h_75,popt_h_100)

In [ ]:
file = open(f"{name}_{temperature}_data.txt","w")

file.write("Angle"+"    ")
file.write("0%"+"    ")
file.write("25%"+"    ")
file.write("50%"+"    ")
file.write("75%"+"    ")
file.write("100%")
file.write("\n")

for i in range (len(plot_h_0)):
    file.write(f"{plot_h_0[i,0]}"+"    ")
    file.write("%.5f"%(plot_h_0[i,1])+" ")
    file.write("%.5f"%(plot_h_25[i,1])+" ")
    file.write("%.5f"%(plot_h_50[i,1])+" ")
    file.write("%.5f"%(plot_h_75[i,1])+" ")
    file.write("%.5f"%(plot_h_100[i,1]))
    file.write("\n")
    
file.close()

In [ ]:
file = open(f"{name}_{temperature}_fitting.txt","w")

file.write("Fitting format: a*exp(-(x-x0)^2/(2*sigma^2))+c")
file.write("\n")

file.write("Strain"+"    ")
file.write("a"+"    ")
file.write("x0"+"    ")
file.write("sigma"+"    ")
file.write("c"+"    ")
file.write("\n")

id_ = 0
for strain in [popt_h_0,popt_h_25,popt_h_50,popt_h_75,popt_h_100]:
    file.write(f"{id_*25}%"+"    ")
    id_ +=1
    for i in range (4):
        file.write("%.8f"%(strain[i])+"    ")
    file.write("\n")
    
file.close()

# D

In [ ]:
name="pdkcd"
temperature = "1.2"
sname = "d"
bname = "PDKCD"

chain_numbers = 48
d_of_p = 32 # degree of polymerization

splited_trj1 = read_files(f"s0_ta-2.trj")
splited_trj2 = read_files(f"s5_ta-2.trj")
splited_trj3 = read_files(f"s10_ta.trj")
splited_trj4 = read_files(f"s15_ta.trj")
splited_trj5 = read_files(f"s20_ta.trj")

In [ ]:
plot_d_0,popt_d_0 = get_plots(splited_trj1,-1,0)
plot_d_25,popt_d_25 = get_plots(splited_trj2,-1,25)
plot_d_50,popt_d_50 = get_plots(splited_trj3,-1,50)
plot_d_75,popt_d_75 = get_plots(splited_trj4,-1,75)
plot_d_100,popt_d_100 = get_plots(splited_trj5,-1,100)

In [ ]:
comparsion(plot_d_0,plot_d_25,plot_d_50,plot_d_75,plot_d_100,popt_d_0,popt_d_25,popt_d_50,popt_d_75,popt_d_100)

In [ ]:
file = open(f"{name}_{temperature}_data.txt","w")

file.write("Angle"+"    ")
file.write("0%"+"    ")
file.write("25%"+"    ")
file.write("50%"+"    ")
file.write("75%"+"    ")
file.write("100%")
file.write("\n")

for i in range (len(plot_m_0)):
    file.write(f"{plot_m_0[i,0]}"+"    ")
    file.write("%.5f"%(plot_d_0[i,1])+" ")
    file.write("%.5f"%(plot_d_25[i,1])+" ")
    file.write("%.5f"%(plot_d_50[i,1])+" ")
    file.write("%.5f"%(plot_d_75[i,1])+" ")
    file.write("%.5f"%(plot_d_100[i,1]))
    file.write("\n")
    
file.close()

In [ ]:
file = open(f"{name}_{temperature}_fitting.txt","w")

file.write("Fitting format: a*exp(-(x-x0)^2/(2*sigma^2))+c")
file.write("\n")

file.write("Strain"+"    ")
file.write("a"+"    ")
file.write("x0"+"    ")
file.write("sigma"+"    ")
file.write("c"+"    ")
file.write("\n")

id_ = 0
for strain in [popt_d_0,popt_d_25,popt_d_50,popt_d_75,popt_d_100]:
    file.write(f"{id_*25}%"+"    ")
    id_ +=1
    for i in range (4):
        file.write("%.8f"%(strain[i])+"    ")
    file.write("\n")
    
file.close()

## 0%

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)
ins = ax.inset_axes([135.4,1.8,42,2.2],transform=ax.transData)

ax.scatter(plot_m_0[:,0],plot_m_0[:,1]*100,label="PDKCM",linewidths=1)
ax.scatter(plot_p_0[:,0],plot_p_0[:,1]*100,label="PDKCP",linewidths=1)
ax.scatter(plot_h_0[:,0],plot_h_0[:,1]*100,label='PDKCH',linewidths=1)
ax.scatter(plot_d_0[:,0],plot_d_0[:,1]*100,label='PDKCD',linewidths=1)


ax.set_xlabel("Dihedral angle ($^\circ$)",size=15)
ax.set_ylabel("Probability (%)",size=15)


ax.set_xticks(np.arange(0,180,step=30))
ax.set_xlim(-4,184)
ax.set_ylim(-0.2,4.2)

ax.legend(loc="upper left",fontsize="x-large")
    
def gaus(x,a,x0,sigma,c):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))+c

ins.plot(plot_m_0[:,0],gaus(plot_m_0[:,0],*popt_m_0)*100,label='PDKCM',linewidth=2)
ins.plot(plot_p_0[:,0],gaus(plot_p_0[:,0],*popt_p_0)*100,label='PDKCP',linewidth=2)
ins.plot(plot_h_0[:,0],gaus(plot_h_0[:,0],*popt_h_0)*100,label='PDKCH',linewidth=2)
ins.plot(plot_d_0[:,0],gaus(plot_d_0[:,0],*popt_d_0)*100,label='PDKCD',linewidth=2)



ins.set_xlabel("Dihedral angle ($^\circ$)",size=10)
ax.set_xticks(np.arange(0,180,step=30))
ins.set_ylim(-0.05,3.05)


#     plt.savefig(f"{name}_{temperature}_all.pdf")
plt.show()